<a href="https://colab.research.google.com/github/jcyeh/dl_scripts/blob/master/fizzbuzz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FizzBuzz in TensorFlow

This Jupyter notebook demonstrates the FizzBuzz code in Python

In [1]:
import numpy as np
import tensorflow as tf
import time

In [2]:
# Represent each input by an array of its binary digits.
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

In [3]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5 == 0: return np.array([0, 0, 1, 0])
    elif i % 3 == 0: return np.array([0, 1, 0, 0])
    else: return np.array([1, 0, 0, 0])

In [4]:
NUM_DIGITS = 10

x_train = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
y_train = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

x_test = np.array([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
y_test = np.array([fizz_buzz_encode(i) for i in range(1, 101)])

In [5]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1000, input_shape=(10,)))
model.add(tf.keras.layers.Activation(tf.nn.relu))
model.add(tf.keras.layers.Dense(4))
model.add(tf.keras.layers.Activation(tf.nn.softmax))

In [6]:
#model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
#model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='Adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, batch_size=64, nb_epoch=120, verbose=0)
end = time.time()
print("Traing Time: {0}s".format(end-start))

Traing Time: 3.2391483783721924s


In [7]:
start = time.time()
result = model.evaluate(x_test, y_test, verbose=0)
end = time.time()
print("Acc: {0:.2f} ".format(result[1]))
print("Evaluation Time: {0}s".format(end-start))

Acc: 0.99 
Evaluation Time: 0.10396957397460938s
